# <b><span style="color:#C21611;">EXTRACCIÓN DE DATOS INMOBILIARIOS EN LIMA-PERÚ</span><b>

En el contexto dinámico del mercado inmobiliario en Lima, Perú, la búsqueda de información precisa y contextualizada se ha vuelto más crucial que nunca. Este proyecto, centrado en la "Extracción de Datos Inmobiliarios en Lima-Perú", no solo se propone compilar datos esenciales sobre bienes raíces, sino que aspira a proporcionar un producto final enriquecido con información adicional. La meta es desarrollar una plataforma completa que no solo proporcione información acerca de precios y ubicaciones, sino que también incluya datos sobre el entorno circundante, tales como la cantidad de parques, supermercados, hospitales y otros elementos fundamentales.

<img src="img1.png" alt="imagen1" style="width:60%;">

### <span style="color:#1557D1;">PROPÓSITO</span><b>

Este proyecto está centrado en la recopilación, procesamiento e integración de datos relacionados con el mercado inmobiliario peruano con el fin de ofrecer una plataforma grande y detallada que sirva como una herramienta valiosa para diversos actores involucrados en el mercado.

### <span style="color:#1557D1;">OBJETIVOS</span><b>

- Desarrollar una plataforma valiosa: Con técnicas avanzadas como web scraping y preprocesamiento de datos, para presentar una visión clara y objetiva del mercado inmobiliario en Lima.

- Beneficiar a múltiples partes interesadas: El proyecto no solo tiene como objetivo informar, sino también empoderar a compradores, vendedores, agentes inmobiliarios y autoridades urbanísticas con información relevante y actualizada.

- Fomentar la transparencia y toma de decisiones informada: Al proporcionar una fuente rica de datos, el proyecto busca mejorar la transparencia del mercado inmobiliario y facilitar la toma de decisiones basadas en datos sólidos.

Utilizamos la siguiente página: https://www.laencontre.com.pe/venta/casas/lima-departamento/list. 'LaEncontré' es una página donde se realizan publicaciónes de bienes inmobiliarios en el Perú, en este caso realizamos la busqueda es en la provincia de Lima, filtrando las caracteristicas más importantes, las cuales son: precio, ubicación, área, cantidad de dormitorios y baños.

In [12]:
# Librerias
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [30]:
# URL de la página web a scrapear
url = "https://www.laencontre.com.pe/venta/casas/lima-departamento/list"

# Realizar una solicitud GET a la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Analizar el contenido HTML de la página usando BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Obtener la lista de precios
    precios_list = [div.text.strip() for div in soup.find_all('div', attrs={"class": "price"})]
    
    # Obtener la lista de ubicaciones
    ubicaciones_list = [a.text.strip() for a in soup.find_all('a', attrs={"class": "detail-redirection"})]
    
    # Obtener la lista de áreas
    areas_list = [span.text.strip() for span in soup.find_all('span', attrs={"class": "areaBuilt"})]
    
    # Obtener la lista de dormitorios
    dormitorios_list = [span.text.strip() for span in soup.find_all('span', attrs={"class": "rooms"})]
    
    # Obtener la lista de baños
    banos_list = [span.text.strip() for span in soup.find_all('span', attrs={"class": "bathrooms"})]

    # Obtener la lista de descripcion
    descrip_list = [p.text.strip() for p in soup.find_all('p', attrs={"class": "description"})]

    # Crear un DataFrame con las listas
    data = {'Precios': precios_list, 'Ubicaciones': ubicaciones_list, 'Área': areas_list, 'Dormitorios': dormitorios_list, 'Baños': banos_list
          ,'Descripcion':descrip_list }
    df = pd.DataFrame(data)

    # Guardar el DataFrame en un archivo Excel
    df.to_excel('datos_inmobiliarios.xlsx', index=False)

    # Imprimir mensaje de éxito
    print("Datos guardados correctamente en 'datos_inmobiliarios.xlsx'")
else:
    # Imprimir mensaje de error si la solicitud no fue exitosa
    print("Error al acceder a la página. Código de estado:", response.status_code)


Datos guardados correctamente en 'datos_inmobiliarios.xlsx'


In [31]:
pip install pivottablejs

Note: you may need to restart the kernel to use updated packages.


In [32]:
from pivottablejs import pivot_ui

In [47]:
df_1= pd.read_excel('datos_inmobiliarios.xlsx', sheet_name ='Sheet1')


In [48]:
df_1

,Precios,Ubicaciones,Áreas,Dormitorios,Baños,Descripcion
0,"$ 930,000","Casa en Calle Los Libertadores, San Isidro, Perú",300m2,3,3,"A.T. : 300 m2, A.C. : 358 m2.\nDos plantas, 23..."
1,"$ 750,000","Casa en Av. Andrés Aramburú, San Isidro, Perú",360m2,3,2,"A.T. : 360 m2, A.C. : 300 m2.\nCasa como terre..."
2,"$ 350,000","Casa en Pasaje Santa Rosa 865 Chaclacayo, Los...",300m2,4,4,"A.T. : 300 m2, A.C. : 300 m2.\nDos plantas, 13..."
3,"$ 1,250,000","Casa en La Planicie, La Molina, Perú","1,790m2",4,8,"Elegante y amplia residencia moderna, 13 años ..."
4,"$ 387,188","Casa en La Calera De La Merced, Surquillo, Perú",178m2,5,3,"SURQUILLO, LA CALERA DE LA MERCED\nVENTA DE CA..."
5,"$ 250,000","Casa en Punta Corrientes, Cerro Azul, Perú",220m2,4,5,CASA DE PLAYA en CONDOMINIO PLAYA PUNTA CORRIE...
6,"$ 1,220,000","Casa en La Planicie, La Molina, Perú","1,124m2",4,4,"En la zona más tranquila de La Planicie, prime..."
7,"$ 247,000","Casa en Punta Corrientes, Cerro Azul, Perú",220m2,4,5,Se vende casa en punta corrientes.\nMuy buena ...
8,"$ 225,000","Casa en Punta Corrientes, Cerro Azul, Perú",220m2,4,5,CASA DE PLAYA en CONDOMINIO PLAYA PUNTA CORRIE...
9,"$ 2,100,000","Casa en Rinconada Alta, La Molina, Perú","2,850m2",4,7,Amplia residencia en Rinconada Alta. 4 dormito...


In [50]:
pivot_ui(df_1)